In [ ]:
# | default_exp benchmark.tsdataset

In [3]:
# | export
import os
from typing import Generator, Tuple

import numpy as np
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe

In [4]:
# | export


class TimeSeriesBenchmarkDataset:
    def __init__(self, dataset_dir: str = "data/"):
        """
        Initialize the dataset benchmark loader with dataset directory.

        :param dataset_dir: Path where datasets are stored.
        """
        self.dataset_dir = dataset_dir

        # Categorizing datasets based on tasks
        self.task_datasets = {
            "short_term_forecast": [],
            "long_term_forecast": [
                "electricity",
                "exchange_rate",
                "weather",
                "illness",
                "ETT-small",
            ],  # "traffic",
            "classification": [
                "EthanolConcentration",
                "FaceDetection",
                "Handwriting",
                "JapaneseVowels",
                "PEMS-SF",
                "SelfRegulationSCP1",
                "SelfRegulationSCP2",
                "SpokenArabicDigits",
                "UWaveGestureLibrary",
            ],
            "anomaly_detection": ["MSL", "PSM", "SMD", "SMAP", "SWaT"],
        }

    def to_long_format(self, df: pd.DataFrame, dataset_name: str) -> pd.DataFrame:
        """
        Convert dataset to long format with columns [unique_id, ds, y].

        :param df: Original dataset
        :param dataset_name: Name of the dataset
        :return: Reformatted DataFrame
        """
        df = df.reset_index()
        df = df.melt(id_vars=["index"], var_name="ds", value_name="y")
        df.rename(columns={"index": "unique_id"}, inplace=True)
        df["unique_id"] = df["unique_id"].astype(str)
        return df

    def load_dataset(self, dataset_name: str) -> pd.DataFrame:
        """
        Load a dataset from the local directory and return it in long format.

        :param dataset_name: Name of the dataset to load.
        :return: DataFrame with columns [unique_id, ds, y].
        """
        dataset_path = os.path.join(self.dataset_dir, dataset_name)

        try:
            # CSV-based datasets (Forecasting)
            if dataset_name in [
                "electricity",
                "exchange_rate",
                "traffic",
                "weather",
                "illness",
                "ETT-small",
            ]:
                csv_file = os.path.join(dataset_path, os.listdir(dataset_path)[0])
                df = pd.read_csv(csv_file)
                df.rename(columns={"date": "ds"}, inplace=True)
                df = df[["unique_id", "ds", "y"]]
                df["unique_id"] = df["unique_id"].astype(str)
                df["ds"] = pd.to_datetime(df.ds)
                return df

            # .TS file-based datasets (Classification)
            elif dataset_name in self.task_datasets["classification"]:
                train_file = os.path.join(dataset_path, f"{dataset_name}_TRAIN.ts")
                test_file = os.path.join(dataset_path, f"{dataset_name}_TEST.ts")

                df_train, y_train = load_from_tsfile_to_dataframe(train_file)
                df_test, y_test = load_from_tsfile_to_dataframe(test_file)

                df_train["label"] = y_train
                df_test["label"] = y_test

                df = pd.concat([df_train, df_test], ignore_index=True)

                return df  # Keeping label column for classification

            # .NPY file-based datasets (Anomaly Detection)
            elif dataset_name in ["MSL", "SMD", "SMAP"]:
                train_file = os.path.join(dataset_path, f"{dataset_name}_train.npy")
                test_file = os.path.join(dataset_path, f"{dataset_name}_test.npy")
                df_train = pd.DataFrame(np.load(train_file))
                df_test = pd.DataFrame(np.load(test_file))
                df = pd.concat([df_train, df_test], ignore_index=True)
                return self.to_long_format(df, dataset_name)

            # .CSV-based anomaly detection (PSM, SWaT)
            elif dataset_name in ["PSM", "SWaT"]:
                train_file = os.path.join(dataset_path, "train.csv")
                test_file = os.path.join(dataset_path, "test.csv")
                df_train = pd.read_csv(train_file)
                df_test = pd.read_csv(test_file)
                df = pd.concat([df_train, df_test], ignore_index=True)
                return self.to_long_format(df, dataset_name)

            else:
                print(f"Dataset {dataset_name} not found or not supported.")
                return None

        except Exception as e:
            print(f"Error loading {dataset_name}: {e}")
            return None

    def get_datasets(self, task: str) -> Generator[Tuple[str, pd.DataFrame], None, None]:
        """
        Generator function to yield datasets for a specific task.

        :param task: Task type ('short_term_forecast', 'long_term_forecast', 'classification', 'anomaly_detection').
        :yield: Tuple of dataset name and DataFrame.
        """
        if task not in self.task_datasets:
            raise ValueError(
                f"Invalid task: {task}. Choose from {list(self.task_datasets.keys())}"
            )

        for dataset_name in self.task_datasets[task]:
            df = self.load_dataset(dataset_name)
            if df is not None:
                yield dataset_name, df

In [83]:
# Example

In [1]:
# | hide
from ts.benchmark.tsdataset import TimeSeriesBenchmarkDataset

# Instantiate and initialize the dataset benchmark loader with your custom data directory
dataset_dir = "data/"
tbd = TimeSeriesBenchmarkDataset(dataset_dir=dataset_dir)

# Define the task to get datasets for this specific task type
task = "classification"  # or short_term_forecasting, long_term_forecasting, anomaly_detection

# Fetch all available datasets of the defined task from your custom data directory
datasets = tbd.get_datasets(task)

for dataset_name, df in datasets:
    print("Processing", dataset_name)
    print(df.columns, df.shape)
    break
    # Preprocess and train your model using these time series datasets

Processing EthanolConcentration
Index(['dim_0', 'dim_1', 'dim_2', 'label'], dtype='object') (524, 4)
